## Set up

First, we import PROCESS and this allows us to run it in the notebook.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import process
from process.main import SingleRun


We can now inspect any variable to check its uninitialised value:

In [2]:
print(
    f"Power diverted to the separartrix (p_plasma_separatrix_mw) = {process.data_structure.physics_variables.p_plasma_separatrix_mw}"
)

Power diverted to the separartrix (p_plasma_separatrix_mw) = None


Now, we need to initialise all the variables in PROCESS with their values at a given design point to establish a baseline.
- We run an PROCESS in _evaluation_ mode to evaluate an input file, and output the values from all models.
- A generic tokamak input, called "large tomamak", is used here.

In [3]:
single_run = SingleRun("data/large_tokamak_eval_IN.DAT")
single_run.run()

The IN.DAT file does not contain any obsolete variables.
 
 **************************************************************************************************************
 ************************************************** PROCESS ***************************************************
 ************************************** Power Reactor Optimisation Code ***************************************
 **************************************************************************************************************
 
 Version : 3.1.0
 Git Tag : v3.1.0-563-g02370a41
 Git Branch : 3818-develop-a-notebook-for-demonstrating-process-for-a-live-audience
 Date : 04/09/2025 UTC
 Time : 14:07
 User : graeme
 Computer : oldstar
 Directory : /home/graeme/Projects/PROCESS/examples
 Input : /home/graeme/Projects/PROCESS/examples/data/large_tokamak_eval_IN.DAT
 Run title : generic large tokamak
 Run type : Reactor concept design: Pulsed tokamak model model, (c) UK Atomic Energy Authority
 
 **********************

/home/graeme/Projects/PROCESS/process/init.py:95: UserWarning: tmargmin_cs and tmargmin should not both be specified in IN.DAT tmargmin_cs has been ignored
  check_process(inputs)
/home/graeme/Projects/PROCESS/process/physics.py:7294: RuntimeWarning: divide by zero encountered in scalar divide
  * (neped / n_greenwald) ** -0.174
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
Ratio of central solenoid overall current density at beginning of flat-top / end of flat-top > 1 (|f_j_cs_start_end_flat_top| > 1)
/home/graeme/Projects/PROCESS/process/costs.py:3030: RuntimeWarning: invalid value encountered in sqrt
  annoam = cost_variables.ucoam[cost_variables.lsa - 1] * np.sqrt(
/home/graeme/Projects/PROCESS/process/costs.py:3101: RuntimeWarning: invalid value encountered in sqrt
  annwst = cost_variables.ucwst[cost_variables.lsa - 1] * np.sqrt(
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, 

 
 ************************************ PROCESS found a consistent solution *************************************
 
 Warning in routine OCMMNT :

*Cryostat roof allowance includes uppermost PF coil and outer thermal shield.
*Cryostat floor allowance includes lowermost PF coil, outer thermal shield and gravity support.
This is longer than 110 columns.
 Warning in routine OCMMNT :

*Cryostat roof allowance includes uppermost PF coil and outer thermal shield.
*Cryostat floor allowance includes lowermost PF coil, outer thermal shield and gravity support.
This is longer than 110 columns.
 Warning in routine OCMMNT :

*Cryostat roof allowance includes uppermost PF coil and outer thermal shield.
*Cryostat floor allowance includes lowermost PF coil, outer thermal shield and gravity support.
This is longer than 110 columns.
******************************************** Errors and Warnings *********************************************

 
 ******************************************* End of PROCESS

In [4]:
phys = process.data_structure.physics_variables
geom = process.data_structure.build_variables
curr = process.data_structure.current_drive_variables

IGNITION_TARGET_TP = 3.0e21  # keV·s·m^-3 (for context)

def triple_product_alpha():
    n   = phys.dene
    T   = getattr(phys, "ti", None) or phys.te  # keV
    tau = getattr(phys, "t_alpha_confinement", None)
    return n*T*tau, n, T, tau

# Define operators for the constraints we care about
# Core
WATCH = {
    5:  "Density (Greenwald) ≤",
    24: "Beta ≤",
    68: "P_sep ≤",
    # Lever-specific adds:
    25: "Peak TF field ≤",           # add when turning Bt
    36: "TF coil temp margin ≥",     # add when turning Bt
    30: "Injection power ≤",         # add when turning Pinj
    16: "Net electric power ≥",      # optional plant metric
}
OPS = {
    5:"<=", 24:"<=", 68:"<=",
    25:"<=", 36:">=",
    30:"<=", 16:">=",
}

from process.constraints import ConstraintManager

def signed_residual(cid):
    """Return a unified residual: >=0 = satisfied, <0 = violated."""
    res = ConstraintManager.evaluate_constraint(cid)
    if res is None:
        return None
    r = res.normalised_residual
    op = OPS.get(cid, None)
    if op == ">=":
        return -r         # flip so negative means violated
    elif op == "<=":
        return r          # already negative when violated for '≤' in PROCESS
    elif op == "=":
        return -abs(r)    # distance from zero treated as violation
    else:
        # Fallback if op unknown: return raw and annotate later
        return r

def report(tag="Baseline"):
    TP, n, T, tau = triple_product_alpha()
    print(f"\n=== {tag} ===")
    print(f"Rmajor = {phys.rmajor:.3f} m")
    print(f"n = {n:.3e} m^-3")
    print(f"T = {T:.3e} keV")
    print(f"τ_α = {tau:.3e} s")
    print(f"nTτ_α = {TP:.3e} keV·s·m^-3 (×{TP/IGNITION_TARGET_TP:.2f} Lawson target)")
    print("\nConstraints (negative = violated):")
    for cid, label in WATCH.items():
        s = signed_residual(cid)
        if s is None:
            print(f"  #{cid:>3} {label:<12}: n/a")
        else:
            mark = "✅" if s >= 0 else "❌"
            print(f"  #{cid:>3} {label:<12}: {mark} (score {s:+.3f})")

# Show baseline status
report("Baseline")



=== Baseline ===
Rmajor = 8.000 m
n = 7.747e+19 m^-3
T = 1.230e+01 keV
τ_α = 1.535e+01 s
nTτ_α = 1.463e+22 keV·s·m^-3 (×4.88 Lawson target)

Constraints (negative = violated):
  #  5 Density (Greenwald) ≤: ❌ (score -0.011)
  # 24 Beta ≤      : ✅ (score +0.183)
  # 68 P_sep ≤     : ✅ (score +0.013)
  # 25 Peak TF field ≤: ❌ (score -0.201)
  # 36 TF coil temp margin ≥: ✅ (score +1.990)
  # 30 Injection power ≤: ❌ (score -0.619)
  # 16 Net electric power ≥: ✅ (score +0.004)


In [5]:
process.data_structure.physics_variables.rmajor = 7.9
single_run.run()

dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value


Solving equality constraints using fsolve


dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommodate the WP, forced to minimum value
dr_tf_plasma_case to small to accommod

 
 ************************************ PROCESS found a consistent solution *************************************
 
 Warning in routine OCMMNT :

*Cryostat roof allowance includes uppermost PF coil and outer thermal shield.
*Cryostat floor allowance includes lowermost PF coil, outer thermal shield and gravity support.
This is longer than 110 columns.
 Warning in routine OCMMNT :

*Cryostat roof allowance includes uppermost PF coil and outer thermal shield.
*Cryostat floor allowance includes lowermost PF coil, outer thermal shield and gravity support.
This is longer than 110 columns.
 Warning in routine OCMMNT :

*Cryostat roof allowance includes uppermost PF coil and outer thermal shield.
*Cryostat floor allowance includes lowermost PF coil, outer thermal shield and gravity support.
This is longer than 110 columns.
******************************************** Errors and Warnings *********************************************

 
 ******************************************* End of PROCESS

In [6]:
report("After Rmajor=7.9m")


=== After Rmajor=7.9m ===
Rmajor = 7.900 m
n = 7.901e+19 m^-3
T = 1.208e+01 keV
τ_α = 1.544e+01 s
nTτ_α = 1.473e+22 keV·s·m^-3 (×4.91 Lawson target)

Constraints (negative = violated):
  #  5 Density (Greenwald) ≤: ❌ (score -0.000)
  # 24 Beta ≤      : ✅ (score +0.185)
  # 68 P_sep ≤     : ❌ (score -0.020)
  # 25 Peak TF field ≤: ❌ (score -0.211)
  # 36 TF coil temp margin ≥: ✅ (score +2.071)
  # 30 Injection power ≤: ❌ (score -0.622)
  # 16 Net electric power ≥: ❌ (score -0.030)


In [7]:
process.data_structure.current_drive_variables.p_hcd_primary_extra_heat_mw = 50.0
single_run.models.current_drive.cudriv()
single_run.models.physics.physics()
single_run.models.power.plant_electric_production()


In [8]:
report("hcd_primary_extra_heat_mw = 10 MW")


=== hcd_primary_extra_heat_mw = 10 MW ===
Rmajor = 7.900 m
n = 7.901e+19 m^-3
T = 1.208e+01 keV
τ_α = 1.544e+01 s
nTτ_α = 1.473e+22 keV·s·m^-3 (×4.91 Lawson target)

Constraints (negative = violated):
  #  5 Density (Greenwald) ≤: ❌ (score -0.000)
  # 24 Beta ≤      : ✅ (score +0.185)
  # 68 P_sep ≤     : ❌ (score -0.170)
  # 25 Peak TF field ≤: ❌ (score -0.211)
  # 36 TF coil temp margin ≥: ✅ (score +2.071)
  # 30 Injection power ≤: ❌ (score -0.747)
  # 16 Net electric power ≥: ✅ (score +0.095)


In [9]:
process.data_structure.physics_variables.bt = 10.0
single_run.models.physics.physics()
single_run.models.current_drive.cudriv()
single_run.models.physics.physics()
single_run.models.power.plant_electric_production()
report("After adding W impurity 5e-5")


=== After adding W impurity 5e-5 ===
Rmajor = 7.900 m
n = 7.901e+19 m^-3
T = 1.208e+01 keV
τ_α = 3.320e+01 s
nTτ_α = 3.169e+22 keV·s·m^-3 (×10.56 Lawson target)

Constraints (negative = violated):
  #  5 Density (Greenwald) ≤: ❌ (score -0.720)
  # 24 Beta ≤      : ✅ (score +0.273)
  # 68 P_sep ≤     : ✅ (score +0.208)
  # 25 Peak TF field ≤: ❌ (score -0.211)
  # 36 TF coil temp margin ≥: ✅ (score +2.071)
  # 30 Injection power ≤: ✅ (score +0.391)
  # 16 Net electric power ≥: ❌ (score -1.043)
